In [1]:
#Importing packages

In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.io as io

In [3]:
#Loading ChEMBL affinity data

In [4]:
thresh = pd.read_csv('/home/daniele/Desktop/plip/map/chembl_23_thresh.csv', sep=',', usecols=['TAR_CHEMBL','LIG_CHEMBL','THRESHOLD','VALUE'], dtype= {'TAR_CHEMBL': np.str,'LIG_CHEMBL': np.str, 'THRESHOLD': np.float, 'VALUE':np.float})

In [5]:
thresh_sorted = thresh.sort_values(['TAR_CHEMBL','LIG_CHEMBL', 'THRESHOLD'])

In [6]:
#Defining 4 thresholds per each drug=target couple

In [7]:
new_c = ['5.5', '6.5', '7.5', '8.5']

In [8]:
for tt in new_c:
    thresh_sorted[tt] = 0.0

In [9]:
df_to_list=thresh_sorted.values.tolist()

In [10]:
list2 = []
for n in df_to_list:
    list2 += [[n[0], n[1]]]

In [11]:
set_td = [list(item) for item in set(tuple(row) for row in list2)]

In [12]:
for td in set_td:
    td.extend([0.0, 0.0, 0.0, 0.0])

In [ ]:
for td in set_td[421176:]:
    for df in df_to_list:
        if (td[0], td[1]) == (df[0], df[1]):
            td[(int(df[2]-3.5))] = df[3]
            if (df[0], df[1]) == (df_to_list[df_to_list.index(df)+1][0], df_to_list[df_to_list.index(df)+1][1]):
                print (set_td.index(td))
            else:
                continue

421176
421176
421176
421177
421177
421177
421178
421178
421178
421179
421179
421179
421180
421180
421180
421181
421181
421181
421182
421182
421182
421183
421183
421183
421184
421184
421184
421185
421185
421185
421186
421186
421187
421187
421187
421188
421188
421188
421189
421189
421189
421190
421190
421190
421191
421191
421191
421192
421192
421192
421193
421193
421193
421194
421194
421194
421195
421195
421195
421196
421196
421196
421197
421197
421197
421198
421198
421198
421199
421199
421199
421200
421200
421200
421201
421201
421201
421202
421202
421202
421203
421203
421203
421204
421204
421204
421205
421205
421205
421206
421206
421207
421207
421207
421208
421208
421208
421209
421209
421209
421210
421210
421210
421211
421211
421211
421212
421212
421212
421213
421213
421213
421214
421214
421214
421215
421215
421215
421216
421216
421216
421217
421217
421217
421218
421218
421218
421219
421219
421219
421220
421220
421220
421221
421221
421221
421222
421222
421222
421223
421223
421223
421224

In [ ]:
import csv

with open('/home/daniele/Desktop/plip/sparseflow/new_ic50.txt', 'a') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(set_td)

csvFile.close()

In [ ]:
df_ic50_new = pd.DataFrame(set_td, columns= ['TARGET', 'LIGAND', '5.5', '6.5', '7.5', '8.5'])

In [ ]:
#Filtering affinity data with PLIP drug-target couples

In [ ]:
interaction_patterns = pd.read_csv('/home/daniele/Desktop/plip/map/interaction_patterns.txt' , sep=':', usecols=['BIN', 'TAR_PDB', 'LIG_PDB', 'CHAIN', 'POSITION'], dtype= {'BIN': np.int, 'TAR_PDB': np.str, 'LIG_PDB': np.str, 'CHAIN': np.str, 'POSITION': np.int} )

In [ ]:
pdb_to_chembl=pd.read_csv('/home/daniele/Desktop/plip/map/src1src3.txt' , sep='\t', usecols=['LIG_CHEMBL','LIG_PDB'], dtype= {'LIG_CHEMBL': np.str, 'LIG_PDB': np.str} )

In [ ]:
lig_mapped = pd.merge(interaction_patterns, pdb_to_chembl, on= [ 'LIG_PDB'])

In [ ]:
pdb_to_uniprot=pd.read_csv('/home/daniele/Desktop/plip/map/pdb_chain_uniprot.tsv' , sep='\t', usecols=['TAR_PDB','CHAIN', 'TAR_UNIPROT'], dtype= {'PDB': np.str, 'CHAIN': np.str, 'TAR_UNIPROT': np.str} )

In [ ]:
pdb_to_uniprot['TAR_PDB'] = map(lambda x: x.upper(), pdb_to_uniprot['TAR_PDB'])

In [ ]:
tar_pdb_uniprot = pd.merge(lig_mapped, pdb_to_uniprot, on= [ 'TAR_PDB', 'CHAIN'])

In [ ]:
uniprot_to_chembl=pd.read_csv('/home/daniele/Desktop/plip/map/chembl_uniprot_mapping.txt' , sep='\t', usecols=['TAR_UNIPROT','TAR_CHEMBL'], dtype= {'TAR_UNIPROT': np.str,'TAR_CHEMBL': np.str} )

In [ ]:
lig_tar_mapped = pd.merge(tar_pdb_uniprot, uniprot_to_chembl, on= [ 'TAR_UNIPROT'])

In [ ]:
plip_cases = pd.DataFrame()

In [ ]:
plip_cases = lig_tar_mapped[['LIG_CHEMBL', 'TAR_CHEMBL']]

In [ ]:
plip_cases.columns= ['LIGAND', 'TARGET']

In [ ]:
thresh_plip = pd.merge(df_ic50_new, plip_cases, on= [ 'LIGAND', 'TARGET'])

In [ ]:
#Creating columns for final matrix (4 thresholds per protein)

In [ ]:
ic50_plip = pd.DataFrame()

In [ ]:
headers = ['LIGAND']

In [ ]:
list_target_set = set(thresh_plip['TARGET'])

In [ ]:
sorted_target = []
for x in sorted(list_target_set):
    sorted_target += [x]

In [ ]:
list_prot_thresh = []
for p in sorted_target:
    list_prot_thresh += p+'_5.5', p+'_6.5', p+'_7.5', p+'_8.5'

In [ ]:
for t in list_prot_thresh:
    headers += [t]

In [ ]:
#Preparing columns (sorted compounds)

In [ ]:
list_lig = thresh_plip['LIGAND']

In [ ]:
set_lig = set(list_lig)
len(set_lig)

In [ ]:
set_comp = []
for p in set_lig:
    set_comp += [p]

In [ ]:
sorted_comp = []
for x in sorted(set_comp):
    sorted_comp += [x]

In [ ]:
#Matrix generation

In [ ]:
lista=[]
l = [0.0]*1256
for c in sorted_comp:
    l.insert(0, c)
    lista += [l]
    l = [0.0]*1256

In [ ]:
ic50_plip = pd.DataFrame(lista)

In [ ]:
ic50_plip.columns= headers

In [ ]:
#Filling the matrix

In [ ]:
ic50_plip_list=ic50_plip.values.tolist()

In [ ]:
thresh_plip_list= thresh_plip.values.tolist()

In [ ]:
list_tar= list(ic50_plip.columns.values)

In [ ]:
li = [len(ic50_plip['LIGAND'])]
mi = range(len(thresh_plip_list))
ni = range(len(list_tar))
for l in li:
    for m in mi:
        for n in ni:
            if ic50_plip_list[l][0] == thresh_plip_list[m][1]:
                if thresh_plip_list[m][0]+'_5.5' == list_tar[n]:
                    ic50_plip_list[l][list_tar.index(list_tar[n])] = thresh_plip_list[m][2]
                    ic50_plip_list[l][list_tar.index(list_tar[n+1])] = thresh_plip_list[m][3]
                    ic50_plip_list[l][list_tar.index(list_tar[n+2])] = thresh_plip_list[m][4]
                    ic50_plip_list[l][list_tar.index(list_tar[n+3])] = thresh_plip_list[m][5]

In [ ]:
ic50_from_plip= pd.DataFrame(ic50_plip_list)

In [ ]:
del ic50_from_plip['0']
del ic50_from_plip['Unnamed: 0']

In [ ]:
y = sp.coo_matrix(ic50_from_plip.values)

In [ ]:
io.mmwrite('/home/daniele/Desktop/plip/sparseflow/ic50_plip_sparse.mtx', y)